In [ ]:
# Required packages for this notebook:
##%pip install --quiet google-genai google-adk python-dotenv

import os
from dotenv import load_dotenv
import sys

# Load environment variables from .env file explicitly
env_path = os.path.join(os.getcwd(), ".env")
print(f"Looking for .env at: {env_path}")
print(f"File exists: {os.path.exists(env_path)}")
load_dotenv(dotenv_path=env_path, override=True)

import numpy as np 
import pandas as pd 

# ADK / GenAI imports
from google import genai, adk
from google.genai import types
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search

# Retrieve API key from environment
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")


if not GEMINI_API_KEY:
    print("GEMINI_API_KEY not found in environment.")
    
else:
    print("GEMINI_API_KEY found in environment.")
    # Configure the Google SDK with the API key
    client = genai.Client(api_key=GEMINI_API_KEY)
    print("Google GenAI SDK configured with API key.")



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking for .env at: /Users/school/Documents/teaching_agents/.env
File exists: True


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GEMINI_API_KEY found in environment.
Google GenAI SDK configured with API key.


In [4]:


from google.adk.runners import Runner

# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("Helper functions defined.")

retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

Helper functions defined.


In [5]:
# Define scope levels for state keys (following best practices)
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")
from typing import Any, Dict, List, Optional
import time
import json

# File used to persist user notes/state. Can override via NOTES_FILE env var.
NOTES_FILE = os.environ.get("NOTES_FILE", "notes.json")

def _read_notes() -> Dict[str, Any]:
    try:
        with open(NOTES_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}
    except Exception as e:
        print(f"Error reading {NOTES_FILE}: {e}")
        return {}

def _write_notes(data: Dict[str, Any]) -> None:
    try:
        with open(NOTES_FILE, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
    except Exception as e:
        print(f"Error writing {NOTES_FILE}: {e}")

def save_userinfo(
    tool_context: Any,
    user_name: str,
    learned_topics: Optional[List[Dict[str, Any]]] = None,
    overall_progress: Optional[float] = None,
    progress_note: Optional[str] = None,
) -> Dict[str, Any]:
    now = int(time.time())
    # Update runtime state for backward compatibility
    tool_context.state["user:name"] = user_name
    tool_context.state["user:learned_topics"] = learned_topics or []
    if overall_progress is not None:
        tool_context.state["user:overall_progress"] = float(overall_progress)
    if progress_note:
        tool_context.state["user:progress_note"] = progress_note[:300]
    tool_context.state["user:last_update"] = now

    # Persist to notes.json keyed by USER_ID
    notes = _read_notes()
    user_entry = notes.get(USER_ID, {})
    user_entry["user_name"] = user_name
    user_entry["learned_topics"] = learned_topics or user_entry.get("learned_topics", [])
    if overall_progress is not None:
        user_entry["overall_progress"] = float(overall_progress)
    if progress_note:
        user_entry["progress_note"] = progress_note[:300]
    user_entry["last_update"] = now
    notes[USER_ID] = user_entry
    _write_notes(notes)
    return {"status": "success", "updated_at": now}

def retrieve_userinfo(tool_context: Any) -> Dict[str, Any]:
    # Prefer persisted data in notes.json, fall back to runtime state
    notes = _read_notes()
    entry = notes.get(USER_ID, {})
    return {
        "status": "success",
        "user_name": entry.get("user_name") or tool_context.state.get("user:name"),
        "learned_topics": entry.get("learned_topics", tool_context.state.get("user:learned_topics", [])),
        "overall_progress": entry.get("overall_progress") or tool_context.state.get("user:overall_progress"),
        "progress_note": entry.get("progress_note") or tool_context.state.get("user:progress_note"),
        "last_update": entry.get("last_update") or tool_context.state.get("user:last_update"),
    }

def update_topic(tool_context: Any, topic_id: str, topic_name: str, proficiency: float) -> Dict[str, Any]:
    notes = _read_notes()
    user_entry = notes.get(USER_ID, {})
    topics = user_entry.get("learned_topics", tool_context.state.get("user:learned_topics", []))
    for t in topics:
        if t.get("topic_id") == topic_id:
            t["topic_name"] = topic_name
            t["proficiency"] = float(proficiency)
            t["last_practiced"] = int(time.time())
            break
    else:
        topics.append({
            "topic_id": topic_id,
            "topic_name": topic_name,
            "proficiency": float(proficiency),
            "last_practiced": int(time.time())
        })
    user_entry["learned_topics"] = topics
    user_entry["last_update"] = int(time.time())
    notes[USER_ID] = user_entry
    _write_notes(notes)
    # Keep runtime state in sync
    tool_context.state["user:learned_topics"] = topics
    tool_context.state["user:last_update"] = user_entry["last_update"]
    return {"status": "success", "topic_id": topic_id}

APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Create an agent with session state tools
teacher_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", api_key=GEMINI_API_KEY, retry_options=retry_config),
    name="TeacherBot",
    instruction="""Teach the user know their level and learning style. Explain the desired content to the user starting with simple concepts and adding depth.
    Tools for managing user context:
    * To record learning progress use `save_userinfo` tool. 
    * To fetch previous learning progress use `retrieve_userinfo` tool.
    * To access factual resources and information regarding the topic use `google_search` tool.
    * Use the `GraderBot` sub-agent to assess student learning.""",
    tools=[save_userinfo, retrieve_userinfo, google_search],
)

grader_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", api_key=GEMINI_API_KEY, retry_options=retry_config),
    name="GraderBot",
    instruction="""This agent's role is to honestly assess students' learning by accurately grading the responses to the questions.""",
    tools=[save_userinfo, retrieve_userinfo, update_topic],
)

# Set up runner with logging plugin
from google.adk.plugins.logging_plugin import LoggingPlugin

runner = InMemoryRunner(
    agent=teacher_agent,
    plugins=[LoggingPlugin()],
)

print("Teaching agent and runner configured successfully!")


Teaching agent and runner configured successfully!
